In [1]:
"""
 author: Prashant Prasad Kanth
 date: 20/09/2022 'MM/DD/YYY'
"""

"\n author: Prashant Prasad Kanth\n date: 20/09/2022 'MM/DD/YYY'\n"

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import count, col, when, isnan

In [3]:
spark = SparkSession.builder.appName("Task3").getOrCreate()
sc = spark.sparkContext

In [4]:
csvpaths = ['csv/state-abbreviation-edited.csv', 'csv/state-abbreviation.csv'
           ,'csv/state-area.csv', 'csv/state-population.csv']
dataframes = []
for path in csvpaths:
    df = spark.read.option("header", True).csv(path)
    dataframes.append(df)
    view_name = path.split('/')[-1].split('.csv')[0].replace('-','_')
    df.createOrReplaceTempView(view_name)

# dataframes[0] => state-abbreviation-edited dataframe
# viewname => state_abbreviation_edited
# dataframes[1] => state-abbreviation dataframe
# viewname => state_abbreviation
# dataframes[2] => state-area dataframe
# viewname => state_area
# dataframes[3] => state-population dataframe
# viewname => state_population

#### Q1. Check for null values in each column in all the files and report the null value count for each

In [5]:
for df, path in zip(dataframes, csvpaths):
    print('Null/NaN values for {}'.format(path))
    df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

Null/NaN values for csv/state-abbreviation-edited.csv
+-----+------------+
|state|abbreviation|
+-----+------------+
|    0|           0|
+-----+------------+

Null/NaN values for csv/state-abbreviation.csv
+-----+------------+
|state|abbreviation|
+-----+------------+
|    0|           0|
+-----+------------+

Null/NaN values for csv/state-area.csv
+-----+----+
|state|area|
+-----+----+
|    0|   0|
+-----+----+

Null/NaN values for csv/state-population.csv
+-----+----+----+----------+
|state|ages|year|population|
+-----+----+----+----------+
|    0|   0|   0|        20|
+-----+----+----+----------+



#### Q2. Report the "total" population of New Jersey from 2001-2010

In [6]:
# dataframes[3].filter((col("state") == "NJ") & (col("year").cast('int').between(2001, 2010))).show()

In [7]:
query = """
        SELECT * 
        FROM state_population
        WHERE state = 'NJ'
        and year between 2001 and 2010
        and ages = 'total'
        order by year
        """
sqlDF = spark.sql(query)
sqlDF.show()

+-----+-----+----+----------+
|state| ages|year|population|
+-----+-----+----+----------+
|   NJ|total|2001|   8492671|
|   NJ|total|2002|   8552643|
|   NJ|total|2003|   8601402|
|   NJ|total|2004|   8634561|
|   NJ|total|2005|   8651974|
|   NJ|total|2006|   8661679|
|   NJ|total|2007|   8677885|
|   NJ|total|2008|   8711090|
|   NJ|total|2009|   8755602|
|   NJ|total|2010|   8802707|
+-----+-----+----+----------+



#### Q3. For the year of 199x (x being the last digit of your NetId), merge “states-area.csv” and “state-population.csv” to get the state wide area and population for the desired year, and compute the state wide area per person for each state. [last digit of netid = 1], `year=1991`

In [8]:
query = """
        SELECT sp.year
              ,sp.state
              ,sa.area
              ,sp.population
              ,round(INT(sa.area)/INT(sp.population), 5) as area_per_person
        FROM state_abbreviation sab
            ,state_population sp
            ,state_area sa
        WHERE 1=1
        and sab.abbreviation = sp.state
        and sab.state = sa.state
        and sp.year = 1991
        and sp.ages = 'total'
        """
sqlDF = spark.sql(query)
rows = sqlDF.count()
print(f"rows: {rows}")
sqlDF.show(rows)

rows: 41
+----+-----+------+----------+---------------+
|year|state|  area|population|area_per_person|
+----+-----+------+----------+---------------+
|1991|   AL| 52423|   4099156|        0.01279|
|1991|   AK|656425|    570193|        1.15123|
|1991|   AZ|114006|   3788576|        0.03009|
|1991|   AR| 53182|   2383144|        0.02232|
|1991|   CA|163707|  30470736|        0.00537|
|1991|   CO|104100|   3387119|        0.03073|
|1991|   CT|  5544|   3302895|        0.00168|
|1991|   DE|  1954|    683080|        0.00286|
|1991|   DC|    68|    600870|         1.1E-4|
|1991|   FL| 65758|  13369798|        0.00492|
|1991|   GA| 59441|   6653005|        0.00893|
|1991|   HI| 10932|   1136754|        0.00962|
|1991|   ID| 83574|   1041316|        0.08026|
|1991|   IL| 57918|  11568964|        0.00501|
|1991|   IN| 36420|   5616388|        0.00648|
|1991|   IA| 56276|   2797613|        0.02012|
|1991|   KS| 82282|   2498722|        0.03293|
|1991|   KY| 40411|   3722328|        0.01086|
|199

#### Q4. For the year of 199x (x being the last digit of your NetId), merge “states-area.csv” and “state-population.csv” with the help of “state-abbrievations-edited.csv” to get the state wide area and population for the desired year, and compute the state wide area per person for each state. [last digit of netid = 1], `year=1991`

In [9]:
query = """
        SELECT sp.year
              ,sp.state
              ,sa.area
              ,sp.population
              ,round(INT(sa.area)/INT(sp.population), 5) as area_per_person
        FROM state_abbreviation_edited sae
            ,state_population sp
            ,state_area sa
        WHERE 1=1
        and sae.abbreviation = sp.state
        and sae.state = sa.state
        and sp.year = 1991
        and sp.ages = 'total'
        """
sqlDF = spark.sql(query)
rows = sqlDF.count()
print(f"rows: {rows}")
sqlDF.show(rows)

rows: 42
+----+-----+------+----------+---------------+
|year|state|  area|population|area_per_person|
+----+-----+------+----------+---------------+
|1991|   AL| 52423|   4099156|        0.01279|
|1991|   AK|656425|    570193|        1.15123|
|1991|   AZ|114006|   3788576|        0.03009|
|1991|   AR| 53182|   2383144|        0.02232|
|1991|   CA|163707|  30470736|        0.00537|
|1991|   CO|104100|   3387119|        0.03073|
|1991|   CT|  5544|   3302895|        0.00168|
|1991|   DE|  1954|    683080|        0.00286|
|1991|   DC|    68|    600870|         1.1E-4|
|1991|   FL| 65758|  13369798|        0.00492|
|1991|   GA| 59441|   6653005|        0.00893|
|1991|   HI| 10932|   1136754|        0.00962|
|1991|   ID| 83574|   1041316|        0.08026|
|1991|   IL| 57918|  11568964|        0.00501|
|1991|   IN| 36420|   5616388|        0.00648|
|1991|   IA| 56276|   2797613|        0.02012|
|1991|   KS| 82282|   2498722|        0.03293|
|1991|   KY| 40411|   3722328|        0.01086|
|199

In [10]:
spark.stop()